In [1]:
%cd ..
%pwd

/Users/areshytko/repos/productivity-dashboard


'/Users/areshytko/repos/productivity-dashboard'

## Load data

In [2]:
from dashboard.load import load
import dashboard.config as config 
from dashboard.gheets import ManualFlow

import json

/Users/areshytko/repos/productivity-dashboard/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
flow = ManualFlow()
url = flow.get_url()
with open('notebooks/url.json', 'w') as wf:
    json.dump(url, wf)

In [4]:
creds = flow.get_google_token()

In [5]:
data = load(
    credentials=creds,
    pomodoros_spreadsheet_id=config.POMODOROS_SPREADSHEET_ID,
    pomodoros_range=config.POMODOROS_RANGE,
    activities_spreadsheet_id=config.ACTIVITIES_SPREADSHEET_ID,
    activities_range=config.ACTIVITIES_RANGE
)

In [6]:
data.df.head()

,Week,Date,Activity,Comment,Pomodoros,Planned,learn,do,career,green,red,personal,hobby,society,life,health,Parent
0,2,2021-01-06,Learn/System Design,None,2.0,4.0,True,False,True,True,False,False,False,False,False,False,
1,2,2021-01-06,Learn/Golang,None,2.0,2.0,True,False,True,True,False,False,False,False,False,False,
2,2,2021-01-06,Learn/Graph Algorithms,None,2.0,2.0,True,False,False,True,False,False,True,False,False,False,
3,2,2021-01-06,Learn/Personal Development,None,1.0,1.0,True,False,False,False,False,True,False,False,False,False,
4,2,2021-01-06,Do/Algorithms,None,NaN,2.0,False,True,True,True,False,False,False,False,False,False,


## Experimenting

In [10]:
import pandas as pd

(pd.Series([1, 2]) == pd.Series([1, 2])).all()

True

In [27]:
import plotly.express as px

x = px.colors.qualitative.Pastel
x

['rgb(102, 197, 204)',
 'rgb(246, 207, 113)',
 'rgb(248, 156, 116)',
 'rgb(220, 176, 242)',
 'rgb(135, 197, 95)',
 'rgb(158, 185, 243)',
 'rgb(254, 136, 177)',
 'rgb(201, 219, 116)',
 'rgb(139, 224, 164)',
 'rgb(180, 151, 231)',
 'rgb(179, 179, 179)']

In [26]:
len(px.colors.qualitative.Vivid_r)

11

In [73]:
df = data.df.groupby('Activity').agg({
    'Pomodoros': 'sum',
    'Parent': 'max'
}).reset_index()

df

,Activity,Pomodoros,Parent
0,"Become a professional blogger on AI, knowledge...",2.0,
1,Do/Algorithms,3.0,
2,Do/Data Engineering,95.0,
3,Do/Trend Detection,2.0,
4,Do/Work,36.0,
5,Learn/Data Engineering,31.0,
6,Learn/Golang,9.0,
7,Learn/Graph Algorithms,6.0,
8,Learn/How to Write,3.0,"Become a professional blogger on AI, knowledge..."
9,Learn/ML,0.0,


In [74]:
df.loc[df.Activity == 'Learn/Medium Articles', 'Parent'] = 'Learn/How to Write'

In [75]:
df

,Activity,Pomodoros,Parent
0,"Become a professional blogger on AI, knowledge...",2.0,
1,Do/Algorithms,3.0,
2,Do/Data Engineering,95.0,
3,Do/Trend Detection,2.0,
4,Do/Work,36.0,
5,Learn/Data Engineering,31.0,
6,Learn/Golang,9.0,
7,Learn/Graph Algorithms,6.0,
8,Learn/How to Write,3.0,"Become a professional blogger on AI, knowledge..."
9,Learn/ML,0.0,


In [41]:
m = df.groupby('Parent').agg({
    'Activity': lambda x: x.to_list()
}).reset_index()
m = m.loc[m.Parent.str.len() > 0, :].set_index('Parent')
m

,Activity
Parent,
"Become a professional blogger on AI, knowledge management, etc.","[Learn/How to Write, Статья по куневин и дата ..."


In [76]:
def compute_descendants(children, nodes):
    result = {}
    for node in nodes:
        descendants = []
        unprocessed = children.get(node, []).copy()
        while unprocessed:
            child = unprocessed.pop()
            descendants.append(child)
            unprocessed += children.get(child, [])
        result[node] = descendants
    return result

In [65]:

class CollectSubprojects:
    
    def __init__(self, df):
        children = df.groupby('Parent').agg({
            'Activity': lambda x: x.to_list()
        }).reset_index()
        children = children.loc[children.Parent.str.len() > 0, :].set_index('Parent')
        self.children = children.Activity.to_dict()
        self.df = df
        self.descendants = descendants(self.children, df.Activity)
    
    def __call__(self):
        self.df.loc[:, 'Pomodoros_With_Subprojects'] = 0
        self.df.loc[:, 'Root_Project'] = None
        for idx, row in self.df.iterrows():
            descendants = self.descendants[row['Activity']]
            descendants = self.df.Activity.isin(descendants)
            self.df.loc[idx, 'Pomodoros_With_Subprojects'] = self.df.loc[descendants, 'Pomodoros'].sum()
            if not row['Parent']:
                self.df.loc[descendants, 'Root_Project'] = row['Activity']
        return self.df

In [79]:
# collect descendants dict: Activity -> Descendants
# apply:
# compute all pomodoros
# filter top level activities
# for each activity set top level


def compute(df):
    children = df.groupby('Parent').agg({
        'Activity': lambda x: x.to_list()
    }).reset_index()
    children = children.loc[children.Parent.str.len() > 0, :].set_index('Parent')
    children = children.Activity.to_dict()
    descendants_dict = compute_descendants(children, df.Activity)
    
    result = df.copy()
    result.loc[:, 'Pomodoros_With_Subprojects'] = 0
    result.loc[:, 'Root_Project'] = result.Activity
    for idx, row in result.iterrows():
        descendants = descendants_dict[row['Activity']]
        descendants = result.Activity.isin(descendants)
        result.loc[idx, 'Pomodoros_With_Subprojects'] = result.loc[descendants, 'Pomodoros'].sum()
        if not row['Parent']:
            result.loc[descendants, 'Root_Project'] = row['Activity']
    return result

In [80]:
compute(df)

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Root_Project
0,"Become a professional blogger on AI, knowledge...",2.0,,7.0,"Become a professional blogger on AI, knowledge..."
1,Do/Algorithms,3.0,,0.0,Do/Algorithms
2,Do/Data Engineering,95.0,,0.0,Do/Data Engineering
3,Do/Trend Detection,2.0,,0.0,Do/Trend Detection
4,Do/Work,36.0,,0.0,Do/Work
5,Learn/Data Engineering,31.0,,0.0,Learn/Data Engineering
6,Learn/Golang,9.0,,0.0,Learn/Golang
7,Learn/Graph Algorithms,6.0,,0.0,Learn/Graph Algorithms
8,Learn/How to Write,3.0,"Become a professional blogger on AI, knowledge...",3.0,"Become a professional blogger on AI, knowledge..."
9,Learn/ML,0.0,,0.0,Learn/ML


In [115]:
collect = CollectSubprojects(df)

In [117]:
x = df.Activity.map(collect)
x + df.Pomodoros

0      9.0
1      3.0
2     95.0
3      2.0
4     36.0
5     31.0
6      9.0
7      6.0
8      6.0
9      0.0
10     3.0
11    30.0
12    56.0
13     3.0
14    69.0
15     3.0
16     2.0
17     1.0
dtype: float64

In [113]:
x[0]

['Статья по куневин и дата саенс',
 'Learn/How to Write',
 'Learn/Medium Articles']

## Testing

In [6]:
from dashboard.process import SubprojectsProcessor

In [7]:
processor = SubprojectsProcessor(data)
result = processor()

In [8]:
result.df

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Fraction
0,"Become a professional blogger on AI, knowledge...",2,,6,0.016713
1,Do/Algorithms,3,,3,0.008357
2,Do/Data Engineering,95,,95,0.264624
3,Do/Trend Detection,2,,2,0.005571
4,Do/Work,36,,36,0.100279
5,Learn/Data Engineering,31,,31,0.086351
6,Learn/Golang,9,,9,0.025070
7,Learn/Graph Algorithms,6,,6,0.016713
8,Learn/How to Write,3,"Become a professional blogger on AI, knowledge...",3,0.008357
9,Learn/ML,0,,0,0.000000


In [11]:
df = result.df.sort_values('Pomodoros_With_Subprojects', ascending=False)
df

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Fraction
2,Do/Data Engineering,95,,95,0.264624
14,Personal Dashboard,74,,74,0.206128
12,Learn/System Design,56,,56,0.155989
4,Do/Work,36,,36,0.100279
5,Learn/Data Engineering,31,,31,0.086351
11,Learn/Personal Development,30,,30,0.083565
6,Learn/Golang,9,,9,0.025070
0,"Become a professional blogger on AI, knowledge...",2,,6,0.016713
7,Learn/Graph Algorithms,6,,6,0.016713
8,Learn/How to Write,3,"Become a professional blogger on AI, knowledge...",3,0.008357


## Visualization

In [14]:
def highlight_survived(s):
    return ['background-color: green']*len(s) if s.Parent else ['background-color: red']*len(s)

def color_survived(val):
    color = 'green' if val else 'red'
    return f'background-color: {color}'

df.style.apply(highlight_survived, axis=1)

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Fraction
2,Do/Data Engineering,95,,95,0.264624
14,Personal Dashboard,74,,74,0.206128
12,Learn/System Design,56,,56,0.155989
4,Do/Work,36,,36,0.100279
5,Learn/Data Engineering,31,,31,0.086351
11,Learn/Personal Development,30,,30,0.083565
6,Learn/Golang,9,,9,0.025070
0,"Become a professional blogger on AI, knowledge management, etc.",2,,6,0.016713
7,Learn/Graph Algorithms,6,,6,0.016713
8,Learn/How to Write,3,"Become a professional blogger on AI, knowledge management, etc.",3,0.008357


In [15]:
import plotly.express as px


In [32]:
colormap

['rgb(102, 197, 204)',
 'rgb(246, 207, 113)',
 'rgb(248, 156, 116)',
 'rgb(220, 176, 242)',
 'rgb(135, 197, 95)',
 'rgb(158, 185, 243)',
 'rgb(254, 136, 177)',
 'rgb(201, 219, 116)',
 'rgb(139, 224, 164)',
 'rgb(180, 151, 231)',
 'rgb(179, 179, 179)',
 '#1F77B4',
 '#FF7F0E',
 '#2CA02C',
 '#D62728',
 '#9467BD',
 '#8C564B',
 '#E377C2',
 '#7F7F7F',
 '#BCBD22',
 '#17BECF']

In [42]:
class RowStyle:
    
    def __init__(self, row_len: int):
        self.colors = self.color_gen(row_len)
    
    def __call__(self, row):
        return next(self.colors)
    
    def color_gen(self, row_len: int):
        colormap = px.colors.qualitative.Pastel + px.colors.qualitative.D3
        for color in colormap:
            yield [f"background-color: {color}"] * row_len
        while True:
            yield ["background-color: white"] * row_len

In [43]:
df.reset_index(drop=True).style.apply(RowStyle(len(df.columns)), axis=1)

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Fraction
0,Do/Data Engineering,95,,95,0.264624
1,Personal Dashboard,74,,74,0.206128
2,Learn/System Design,56,,56,0.155989
3,Do/Work,36,,36,0.100279
4,Learn/Data Engineering,31,,31,0.086351
5,Learn/Personal Development,30,,30,0.083565
6,Learn/Golang,9,,9,0.025070
7,"Become a professional blogger on AI, knowledge management, etc.",2,,6,0.016713
8,Learn/Graph Algorithms,6,,6,0.016713
9,Learn/How to Write,3,"Become a professional blogger on AI, knowledge management, etc.",3,0.008357


In [44]:
class CellStyle:
    
    def __init__(self):
        self.colors = self.color_gen()
    
    def __call__(self, cell):
        return next(self.colors)
    
    def color_gen(self):
        colormap = px.colors.qualitative.Pastel + px.colors.qualitative.D3
        for color in colormap:
            yield f"background-color: {color}"
        while True:
            yield "background-color: white"

In [47]:
df.reset_index(drop=True).style.applymap(CellStyle(), subset=['Activity'])

,Activity,Pomodoros,Parent,Pomodoros_With_Subprojects,Fraction
0,Do/Data Engineering,95,,95,0.264624
1,Personal Dashboard,74,,74,0.206128
2,Learn/System Design,56,,56,0.155989
3,Do/Work,36,,36,0.100279
4,Learn/Data Engineering,31,,31,0.086351
5,Learn/Personal Development,30,,30,0.083565
6,Learn/Golang,9,,9,0.025070
7,"Become a professional blogger on AI, knowledge management, etc.",2,,6,0.016713
8,Learn/Graph Algorithms,6,,6,0.016713
9,Learn/How to Write,3,"Become a professional blogger on AI, knowledge management, etc.",3,0.008357
